In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
from scipy.stats import linregress
from datetime import datetime
from yelpapi import YelpAPI

# Impor the API key
from config import api_key

In [4]:
finished_url = f"https://api.yelp.com/v3/businesses/north-india-restaurant-san-francisco"
#result_data = requests.get(finished_url).json()
#pprint(result_data)

yelp_api = YelpAPI(api_key)
search_term = 'restaurants'
location = 'New York City, NY'
limit = 50  # maximum number of results to return
daily_results_limit = 12500 # update this by 12500 every day
offset = 0 # update this by 12500 every day

# response = yelp_api.search_query(term=search_term, location=location, limit=limit)
# pprint(response)
results = []
api_call_limit = 10  # total API call limit for the day
calls_made = 0  # counter for API calls made

while offset < daily_results_limit and calls_made < api_call_limit:
    response = yelp_api.search_query(term=search_term, location=location, limit=limit, offset=offset)
    results.extend(response['businesses'])  # Extend the list with new businesses
    offset += limit  # Increment the offset by the limit
    calls_made += 1  # Increment the API call counter
    
    # Print progress
    print(f'Retrieved {len(results)} results with {calls_made} API calls')

    # Avoid hitting rate limits by adding a delay
    time.sleep(1)  # Delay for 1 second

Retrieved 50 results with 1 API calls
Retrieved 100 results with 2 API calls
Retrieved 150 results with 3 API calls
Retrieved 200 results with 4 API calls


HTTPError: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?term=restaurants&location=New+York+City%2C+NY&limit=50&offset=200

In [ ]:
# This code was for testing and looking at how we wanted to pull the data

# print(response['businesses'][0]['name'])
# print(f"{response['businesses'][0]['rating']} ({response['businesses'][0]['review_count']})")
# print(response['businesses'][0]['price'])
# print(response['businesses'][0]['display_phone'])
# print(f"Latitiude: {response['businesses'][0]['coordinates']['latitude']}, Longitude: {response['businesses'][0]['coordinates']['longitude']}")

# test_categories = ''
# for alias in response['businesses'][0]['categories']:
#     test_categories = test_categories + '/' + alias['title']
# print(test_categories)

# test_transactions = ''
# for type in response['businesses'][0]['transactions']:
#     test_transactions = test_transactions + '/' + type
# print(test_transactions)

In [5]:
restaurant_data = []
count = 0
for restaurant in results:
    name = restaurant['name']
    rating = restaurant['rating']
    review_count = restaurant['review_count']
    try:
        price = restaurant['price']
    except KeyError:
        price = '???'
    phone_number = restaurant['display_phone']
    latitiude = restaurant['coordinates']['latitude']
    longitude = restaurant['coordinates']['longitude']

    categories = ''
    x = 0
    for alias in restaurant['categories']:
        if x == 0:
            categories = alias['title']
            x += 1
        else:
            categories = categories + '/' + alias['title']

    transactions = ''
    x = 0
    for type in restaurant['transactions']:
        if x == 0:
            transactions = type
            x += 1
        else:
            transactions = transactions + '/' + type

    restaurant_data.append({'Name':name,
                            'Rating':rating,
                            'Review Count':review_count,
                            'Price':price,
                            'Phone Number':phone_number,
                            'Latitiude':latitiude,
                            'Longitude':longitude,
                            'Categories':categories,
                            'Transactions':transactions})
    count += 1

restaurant_data_df = pd.DataFrame(restaurant_data)
restaurant_data_df.to_csv("data/restaurant_data.csv",mode='a',header=False)